# Orthogonal Fine-Tuning (OFT) Tutorial

## Introduction

Orthogonal Fine-Tuning (OFT) is a parameter-efficient fine-tuning (PEFT) technique that adapts pre-trained models by applying orthogonal transformations to weight matrices. Unlike other PEFT methods like LoRA, OFT preserves the hyperspherical energy – the geometric relationships between neuron activations – ensuring stable and efficient adaptation.

## 1. Understanding Orthogonal Matrices

An orthogonal matrix Q satisfies: Q^T Q = QQ^T = I

Key properties:
- Preserves distances and angles
- Preserves norms: ||Qx|| = ||x||
- Represents rotations and reflections

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from scipy.stats import special_ortho_group

# Generate a random orthogonal matrix
def generate_orthogonal_matrix(dim):
    """Generate a random orthogonal matrix using QR decomposition"""
    random_matrix = np.random.randn(dim, dim)
    q, r = np.linalg.qr(random_matrix)
    return q

# Verify orthogonality
Q = generate_orthogonal_matrix(4)
print("Random orthogonal matrix Q:")
print(Q)
print("\nQ^T @ Q (should be identity):")
print(np.round(Q.T @ Q, 4))
print("\nDeterminant (should be ±1):", np.linalg.det(Q))

## 2. Visualizing Orthogonal Transformations

Let's visualize how orthogonal transformations preserve geometric relationships while rotating the feature space.

In [ ]:
# Create sample 2D data
np.random.seed(42)
n_points = 100
original_data = np.random.randn(n_points, 2)

# Create a 2D rotation matrix (orthogonal)
theta = np.pi / 4  # 45 degrees
rotation_matrix = np.array([
    [np.cos(theta), -np.sin(theta)],
    [np.sin(theta), np.cos(theta)]
])

# Apply orthogonal transformation
transformed_data = original_data @ rotation_matrix.T

# Plotting
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original data
axes[0].scatter(original_data[:, 0], original_data[:, 1], alpha=0.6)
axes[0].set_title('Original Data')
axes[0].set_xlim(-4, 4)
axes[0].set_ylim(-4, 4)
axes[0].grid(True, alpha=0.3)
axes[0].set_aspect('equal')

# Transformed data
axes[1].scatter(transformed_data[:, 0], transformed_data[:, 1], alpha=0.6, color='orange')
axes[1].set_title('After Orthogonal Transformation')
axes[1].set_xlim(-4, 4)
axes[1].set_ylim(-4, 4)
axes[1].grid(True, alpha=0.3)
axes[1].set_aspect('equal')

# Overlay comparison
axes[2].scatter(original_data[:, 0], original_data[:, 1], alpha=0.6, label='Original')
axes[2].scatter(transformed_data[:, 0], transformed_data[:, 1], alpha=0.6, color='orange', label='Transformed')
axes[2].set_title('Comparison: Rotation Preserves Structure')
axes[2].set_xlim(-4, 4)
axes[2].set_ylim(-4, 4)
axes[2].grid(True, alpha=0.3)
axes[2].set_aspect('equal')
axes[2].legend()

plt.tight_layout()
plt.show()

# Verify preservation of distances
original_distances = np.linalg.norm(original_data[0] - original_data[1])
transformed_distances = np.linalg.norm(transformed_data[0] - transformed_data[1])
print(f"Distance between first two points:")
print(f"  Original: {original_distances:.4f}")
print(f"  Transformed: {transformed_distances:.4f}")
print(f"  Preserved: {np.isclose(original_distances, transformed_distances)}")

## 3. OFT Core Implementation

The key insight of OFT is to parameterize weight updates as:

**W' = W × R**

Where:
- W is the original pre-trained weight matrix
- R is an orthogonal matrix (learned during fine-tuning)
- W' is the adapted weight matrix

In [ ]:
class OFTLayer(nn.Module):
    """
    Orthogonal Fine-Tuning layer implementation.
    Applies an orthogonal transformation to the weight matrix.
    """
    def __init__(self, in_features, out_features, rank=16):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.rank = min(rank, min(in_features, out_features))
        
        # Pre-trained weights (frozen)
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.weight.requires_grad = False
        
        # Orthogonal transformation parameters
        # We use Cayley parameterization for stable orthogonal updates
        self.cayley_map = nn.Parameter(torch.zeros(self.rank, self.rank))
        
    def get_orthogonal_matrix(self):
        """
        Compute orthogonal matrix using Cayley transform:
        R = (I - A)(I + A)^(-1)
        where A is skew-symmetric
        """
        # Make skew-symmetric
        A = self.cayley_map - self.cayley_map.t()
        I = torch.eye(self.rank, device=A.device)
        
        # Cayley transform
        R = torch.linalg.solve(I + A, I - A)
        return R
    
    def forward(self, x):
        # Get orthogonal transformation
        R = self.get_orthogonal_matrix()
        
        # For simplicity, apply R to a subspace of the weight matrix
        # In practice, this would be more sophisticated
        W_adapted = self.weight.clone()
        W_adapted[:self.rank, :self.rank] = self.weight[:self.rank, :self.rank] @ R
        
        return x @ W_adapted.t()

# Example usage
oft_layer = OFTLayer(128, 64, rank=16)
input_tensor = torch.randn(32, 128)  # batch_size=32, in_features=128
output = oft_layer(input_tensor)
print(f"Input shape: {input_tensor.shape}")
print(f"Output shape: {output.shape}")
print(f"Trainable parameters: {sum(p.numel() for p in oft_layer.parameters() if p.requires_grad)}")
print(f"Total parameters: {sum(p.numel() for p in oft_layer.parameters())}")

## 4. Comparing OFT with LoRA

Let's implement a simple comparison between OFT and LoRA to understand their differences.

In [ ]:
class LoRALayer(nn.Module):
    """
    Low-Rank Adaptation (LoRA) layer for comparison.
    """
    def __init__(self, in_features, out_features, rank=16):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.rank = rank
        
        # Pre-trained weights (frozen)
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.weight.requires_grad = False
        
        # LoRA parameters
        self.lora_A = nn.Parameter(torch.randn(rank, in_features) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(out_features, rank))
        
    def forward(self, x):
        # Original transformation + low-rank update
        return x @ self.weight.t() + x @ self.lora_A.t() @ self.lora_B.t()

# Compare parameter efficiency
in_features, out_features = 512, 256
rank = 16

oft_layer = OFTLayer(in_features, out_features, rank)
lora_layer = LoRALayer(in_features, out_features, rank)

oft_params = sum(p.numel() for p in oft_layer.parameters() if p.requires_grad)
lora_params = sum(p.numel() for p in lora_layer.parameters() if p.requires_grad)
total_params = in_features * out_features

print("Parameter Efficiency Comparison:")
print(f"Original layer parameters: {total_params:,}")
print(f"OFT trainable parameters: {oft_params:,} ({oft_params/total_params*100:.2f}%)")
print(f"LoRA trainable parameters: {lora_params:,} ({lora_params/total_params*100:.2f}%)")

## 5. Hyperspherical Energy Preservation

A key advantage of OFT is preserving hyperspherical energy, which maintains the angular relationships between features. Let's demonstrate this concept.

In [ ]:
def compute_hyperspherical_energy(features):
    """
    Compute hyperspherical energy as the sum of pairwise angular similarities.
    Lower energy indicates more uniform distribution on the hypersphere.
    """
    # Normalize features to unit sphere
    normalized = features / (torch.norm(features, dim=-1, keepdim=True) + 1e-8)
    
    # Compute pairwise cosine similarities
    similarities = torch.matmul(normalized, normalized.t())
    
    # Hyperspherical energy (Riesz s-energy with s=1)
    n = features.shape[0]
    mask = 1 - torch.eye(n)
    energy = -torch.sum(torch.log(1 - similarities + 1e-8) * mask) / (n * (n - 1))
    
    return energy

# Generate sample features
torch.manual_seed(42)
n_samples = 50
n_features = 64

# Original features
original_features = torch.randn(n_samples, n_features)

# Apply different transformations
# 1. Orthogonal transformation (OFT-style)
Q = torch.tensor(generate_orthogonal_matrix(n_features), dtype=torch.float32)
oft_features = original_features @ Q.t()

# 2. Random transformation (non-orthogonal)
random_matrix = torch.randn(n_features, n_features) * 0.5 + torch.eye(n_features)
random_features = original_features @ random_matrix.t()

# 3. Low-rank update (LoRA-style)
A = torch.randn(16, n_features) * 0.1
B = torch.randn(n_features, 16) * 0.1
lora_features = original_features + original_features @ A.t() @ B.t()

# Compute energies
original_energy = compute_hyperspherical_energy(original_features)
oft_energy = compute_hyperspherical_energy(oft_features)
random_energy = compute_hyperspherical_energy(random_features)
lora_energy = compute_hyperspherical_energy(lora_features)

print("Hyperspherical Energy Comparison:")
print(f"Original features: {original_energy:.4f}")
print(f"After OFT (orthogonal): {oft_energy:.4f} (change: {abs(oft_energy - original_energy):.4f})")
print(f"After random transform: {random_energy:.4f} (change: {abs(random_energy - original_energy):.4f})")
print(f"After LoRA update: {lora_energy:.4f} (change: {abs(lora_energy - original_energy):.4f})")
print("\nOFT preserves hyperspherical energy best!")

## 6. Practical OFT Module for Transformer Models

Let's implement a more practical OFT module that can be integrated into transformer-based models.

In [ ]:
class OFTModule(nn.Module):
    """
    Production-ready OFT module with block-diagonal structure for efficiency.
    """
    def __init__(self, original_module, rank=16, num_blocks=4, alpha=1.0):
        super().__init__()
        self.original_module = original_module
        self.rank = rank
        self.num_blocks = num_blocks
        self.alpha = alpha  # Scaling factor for the adaptation
        
        # Get dimensions from original module
        if isinstance(original_module, nn.Linear):
            self.in_features = original_module.in_features
            self.out_features = original_module.out_features
        else:
            raise ValueError("OFT currently supports only Linear layers")
        
        # Block size for block-diagonal structure
        self.block_size = min(rank, min(self.in_features, self.out_features) // num_blocks)
        
        # Initialize block-diagonal orthogonal parameters
        self.blocks = nn.ParameterList([
            nn.Parameter(torch.zeros(self.block_size, self.block_size))
            for _ in range(num_blocks)
        ])
        
        # Freeze original weights
        for param in self.original_module.parameters():
            param.requires_grad = False
    
    def get_block_diagonal_orthogonal(self):
        """
        Construct block-diagonal orthogonal matrix from individual blocks.
        """
        device = self.blocks[0].device
        total_size = self.block_size * self.num_blocks
        R = torch.zeros(total_size, total_size, device=device)
        
        for i, block_param in enumerate(self.blocks):
            # Apply Cayley transform to each block
            A = block_param - block_param.t()
            I = torch.eye(self.block_size, device=device)
            block_R = torch.linalg.solve(I + A, I - A)
            
            # Place in block-diagonal structure
            start = i * self.block_size
            end = start + self.block_size
            R[start:end, start:end] = block_R
        
        return R
    
    def forward(self, x):
        # Get original output
        original_output = self.original_module(x)
        
        # Apply orthogonal adaptation to a subspace
        R = self.get_block_diagonal_orthogonal()
        adapted_weight = self.original_module.weight.clone()
        
        # Apply rotation to subspace of weights
        subspace_size = R.shape[0]
        if subspace_size <= min(self.out_features, self.in_features):
            W_sub = adapted_weight[:subspace_size, :subspace_size]
            adapted_weight[:subspace_size, :subspace_size] = W_sub @ R
        
        # Compute adapted output
        adapted_output = nn.functional.linear(x, adapted_weight, self.original_module.bias)
        
        # Blend original and adapted outputs
        return (1 - self.alpha) * original_output + self.alpha * adapted_output

# Example: Applying OFT to a pre-trained linear layer
original_layer = nn.Linear(256, 128)
oft_adapted_layer = OFTModule(original_layer, rank=8, num_blocks=4, alpha=0.5)

# Test forward pass
test_input = torch.randn(16, 256)
output = oft_adapted_layer(test_input)

print(f"OFT Module Configuration:")
print(f"  Original layer: Linear({original_layer.in_features}, {original_layer.out_features})")
print(f"  Block size: {oft_adapted_layer.block_size}")
print(f"  Number of blocks: {oft_adapted_layer.num_blocks}")
print(f"  Trainable parameters: {sum(p.numel() for p in oft_adapted_layer.parameters() if p.requires_grad)}")
print(f"  Output shape: {output.shape}")

## 7. Training Example with OFT

Let's demonstrate how to train a model using OFT for a simple classification task.

In [ ]:
# Create a simple dataset
from torch.utils.data import TensorDataset, DataLoader

# Generate synthetic classification data
torch.manual_seed(42)
n_samples = 1000
n_features = 64
n_classes = 10

X = torch.randn(n_samples, n_features)
y = torch.randint(0, n_classes, (n_samples,))

# Split into train and test
train_size = int(0.8 * n_samples)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create dataloaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define a simple model with OFT adaptation
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, use_oft=False):
        super().__init__()
        self.use_oft = use_oft
        
        # Create layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
        # Apply OFT if requested
        if use_oft:
            self.fc1 = OFTModule(self.fc1, rank=8, num_blocks=2)
            self.fc2 = OFTModule(self.fc2, rank=8, num_blocks=2)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create two models: one with OFT, one without
model_oft = SimpleClassifier(n_features, 128, n_classes, use_oft=True)
model_regular = SimpleClassifier(n_features, 128, n_classes, use_oft=False)

# Count parameters
oft_trainable = sum(p.numel() for p in model_oft.parameters() if p.requires_grad)
regular_trainable = sum(p.numel() for p in model_regular.parameters() if p.requires_grad)

print("Model Comparison:")
print(f"Regular model trainable parameters: {regular_trainable:,}")
print(f"OFT model trainable parameters: {oft_trainable:,}")
print(f"Parameter reduction: {(1 - oft_trainable/regular_trainable)*100:.1f}%")

In [ ]:
# Training function
def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    train_losses = []
    test_accuracies = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        train_losses.append(epoch_loss / len(train_loader))
        
        # Testing
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()
        
        accuracy = correct / total
        test_accuracies.append(accuracy)
        
        if (epoch + 1) % 2 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {train_losses[-1]:.4f}, Test Acc: {accuracy:.4f}")
    
    return train_losses, test_accuracies

# Train OFT model
print("Training OFT Model:")
oft_losses, oft_accs = train_model(model_oft, train_loader, test_loader, epochs=10)

# For comparison, we could also train the regular model
# Note: In practice, OFT is used for fine-tuning pre-trained models

## 8. Advantages and Use Cases of OFT

### Key Advantages:

1. **Stability**: Orthogonal transformations prevent gradient explosion/vanishing
2. **Parameter Efficiency**: Fewer trainable parameters than full fine-tuning
3. **Geometric Preservation**: Maintains learned feature relationships
4. **Better Generalization**: Less prone to overfitting on small datasets

### Best Use Cases:

- **Domain Adaptation**: When fine-tuning to a related but different domain
- **Few-Shot Learning**: When training data is limited
- **Multi-Task Learning**: Preserving shared representations across tasks
- **Continual Learning**: Reducing catastrophic forgetting

In [ ]:
# Demonstration: Stability comparison
def analyze_gradient_flow(model, input_data, target):
    """
    Analyze gradient magnitudes through the network.
    """
    model.train()
    criterion = nn.CrossEntropyLoss()
    
    # Forward pass
    output = model(input_data)
    loss = criterion(output, target)
    
    # Backward pass
    loss.backward()
    
    # Collect gradient norms
    gradient_norms = []
    for name, param in model.named_parameters():
        if param.requires_grad and param.grad is not None:
            grad_norm = param.grad.norm().item()
            gradient_norms.append(grad_norm)
            print(f"{name}: gradient norm = {grad_norm:.6f}")
    
    return gradient_norms

# Create test data
test_input = torch.randn(32, n_features)
test_target = torch.randint(0, n_classes, (32,))

print("Gradient Flow Analysis for OFT Model:")
print("="*50)
grad_norms = analyze_gradient_flow(model_oft, test_input, test_target)
print(f"\nAverage gradient norm: {np.mean(grad_norms):.6f}")
print(f"Gradient variance: {np.var(grad_norms):.6f}")
print("\nNote: OFT maintains stable gradients due to orthogonal constraints!")

## Summary

Orthogonal Fine-Tuning (OFT) represents a significant advancement in parameter-efficient fine-tuning:

- **Preserves geometric structure** of pre-trained representations
- **More stable training** compared to unconstrained methods
- **Highly parameter-efficient** while maintaining model capacity
- **Better suited** for tasks requiring preservation of learned features

The orthogonal constraint ensures that fine-tuning rotates the feature space rather than arbitrarily distorting it, leading to more robust and generalizable adaptations.